In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV

In [2]:
from proj1_helpers import *
from implementations import *

In [3]:
labels, features, ids = load_csv_data('train.csv') 

In [4]:
train = pd.read_csv('train.csv', index_col = 0)
train.head(8)

Prediction  DER_mass_MMC  DER_mass_transverse_met_lep  DER_mass_vis  \
Id                                                                           
100000          s       138.470                       51.655        97.827   
100001          b       160.937                       68.768       103.235   
100002          b      -999.000                      162.172       125.953   
100003          b       143.905                       81.417        80.943   
100004          b       175.864                       16.915       134.805   
100005          b        89.744                       13.550        59.149   
100006          s       148.754                       28.862       107.782   
100007          s       154.916                       10.418        94.714   

        DER_pt_h  DER_deltaeta_jet_jet  DER_mass_jet_jet  DER_prodeta_jet_jet  \
Id                                                                              
100000    27.980                 0.910           124.711                2.666   
100001    48.146              -999.000          -999.000             -999.000   
100002    35.635              -999.000          -999.000             -999.000   
100003     0.414              -999.000          -999.000             -999.000   
100004    16.405              -999.000          -999.000             -999.000   
100005   116.344                 2.636           284.584               -0.540   
100006   106.130                 0.733           158.359                0.113   
100007    29.169              -999.000          -999.000             -999.000   

        DER_deltar_tau_lep  DER_pt_tot       ...        PRI_met_phi  \
Id                                           ...                      
100000               3.064      41.928       ...             -0.277   
100001               3.473       2.078       ...             -1.916   
100002               3.148       9.336       ...             -2.186   
100003               3.310       0.414       ...              0.060   
100004               3.891      16.405       ...             -0.871   
100005               1.362      61.619       ...              2.237   
100006               2.941       2.545       ...             -1.443   
100007               2.897       1.526       ...             -1.761   

        PRI_met_sumet  PRI_jet_num  PRI_jet_leading_pt  PRI_jet_leading_eta  \
Id                                                                            
100000        258.733            2              67.435                2.150   
100001        164.546            1              46.226                0.725   
100002        260.414            1              44.251                2.053   
100003         86.062            0            -999.000             -999.000   
100004         53.131            0            -999.000             -999.000   
100005        282.849            3              90.547               -2.412   
100006        294.074            2             123.010                0.864   
100007        187.299            1              30.638               -0.715   

        PRI_jet_leading_phi  PRI_jet_subleading_pt  PRI_jet_subleading_eta  \
Id                                                                           
100000                0.444                 46.062                   1.240   
100001                1.158               -999.000                -999.000   
100002               -2.028               -999.000                -999.000   
100003             -999.000               -999.000                -999.000   
100004             -999.000               -999.000                -999.000   
100005               -0.653                 56.165                   0.224   
100006                1.450                 56.867                   0.131   
100007               -1.724               -999.000                -999.000   

        PRI_jet_subleading_phi  PRI_jet_all_pt  
Id                                              
100000                  -2.475    

In [5]:
train['Prediction'] = train['Prediction'].replace({'s': 1, 'b': -1})


In [6]:
#idx = train[train['PRI_jet_all_pt'] == 0].index
#train = train.drop(idx)


In [7]:
feature_selected = train.drop(columns = ['DER_deltaeta_jet_jet', 
                                         'DER_lep_eta_centrality', 
                                         'DER_prodeta_jet_jet', 
                                         'PRI_jet_leading_eta', 
                                         'DER_mass_jet_jet', 
                                         'PRI_jet_leading_phi',
                                         'PRI_jet_num',
                                        'PRI_jet_subleading_eta',
                                        'PRI_jet_subleading_phi',
                                        'PRI_jet_subleading_pt',
                                        'PRI_lep_eta',
                                        'PRI_lep_phi',
                                        'PRI_met_phi',
                                        'PRI_tau_eta',
                                        'PRI_tau_phi',
                                        'PRI_jet_all_pt',
                                        'PRI_jet_leading_pt'])

In [8]:
feature_selected.count()

Prediction                     250000
DER_mass_MMC                   250000
DER_mass_transverse_met_lep    250000
DER_mass_vis                   250000
DER_pt_h                       250000
DER_deltar_tau_lep             250000
DER_pt_tot                     250000
DER_sum_pt                     250000
DER_pt_ratio_lep_tau           250000
DER_met_phi_centrality         250000
PRI_tau_pt                     250000
PRI_lep_pt                     250000
PRI_met                        250000
PRI_met_sumet                  250000
dtype: int64

In [9]:
median_mass_MMC = np.median(feature_selected.DER_mass_MMC[feature_selected.DER_mass_MMC != -999])

In [10]:
feature_selected['DER_mass_MMC'] = feature_selected['DER_mass_MMC'].replace({-999: median_mass_MMC})

In [11]:
#feature_selected.to_excel('/Users/wentao/Desktop/ML Project 1/feature_selected.xlsx')

In [12]:
feature_selected.describe()

Prediction   DER_mass_MMC  DER_mass_transverse_met_lep  \
count  250000.000000  250000.000000                250000.000000   
mean       -0.314664     120.417434                    49.239819   
std         0.949205      52.859218                    35.344886   
min        -1.000000       9.044000                     0.000000   
25%        -1.000000      95.665000                    19.241000   
50%        -1.000000     112.406000                    46.524000   
75%         1.000000     130.606250                    73.598000   
max         1.000000    1192.026000                   690.075000   

        DER_mass_vis       DER_pt_h  DER_deltar_tau_lep     DER_pt_tot  \
count  250000.000000  250000.000000       250000.000000  250000.000000   
mean       81.181982      57.895962            2.373100      18.917332   
std        40.828691      63.655682            0.782911      22.273494   
min         6.329000       0.000000            0.208000       0.000000   
25%        59.388750      14.068750            1.810000       2.841000   
50%        73.752000      38.467500            2.491500      12.315500   
75%        92.259000      79.169000            2.961000      27.591000   
max      1349.351000    2834.999000            5.684000    2834.999000   

          DER_sum_pt  DER_pt_ratio_lep_tau  DER_met_phi_centrality  \
count  250000.000000         250000.000000           250000.000000   
mean      158.432217              1.437609               -0.128305   
std       115.706115              0.844743                1.193585   
min        46.104000              0.047000               -1.414000   
25%        77.550000              0.883000               -1.371000   
50%       120.664500              1.280000               -0.356000   
75%       200.478250              1.777000                1.225000   
max      1852.462000             19.773000                1.414000   

          PRI_tau_pt     PRI_lep_pt        PRI_met  PRI_met_sumet  
count  250000.000000  250000.000000  250000.000000  250000.000000  
mean       38.707419      46.660207      41.717235     209.797178  
std        22.412081      22.064922      32.894693     126.499506  
min        20.000000      26.000000       0.109000      13.678000  
25%        24.591750      32.375000      21.398000     123.017500  
50%        31.804000      40.516000      34.802000     179.739000  
75%        45.017000      53.390000      51.895000     263.379250  
max       764.408000     560.271000    2842.617000    2003.976000

In [13]:
corr_coef = feature_selected.drop(columns='Prediction').corr(method = 'pearson')
corr_coef

DER_mass_MMC  DER_mass_transverse_met_lep  \
DER_mass_MMC                     1.000000                     0.146156   
DER_mass_transverse_met_lep      0.146156                     1.000000   
DER_mass_vis                     0.805154                     0.190109   
DER_pt_h                         0.035232                    -0.249116   
DER_deltar_tau_lep               0.458074                     0.043251   
DER_pt_tot                       0.015142                     0.017758   
DER_sum_pt                       0.129819                    -0.146837   
DER_pt_ratio_lep_tau             0.064216                     0.349504   
DER_met_phi_centrality           0.041740                    -0.419757   
PRI_tau_pt                       0.255751                    -0.145464   
PRI_lep_pt                       0.321339                     0.310648   
PRI_met                          0.075356                     0.183716   
PRI_met_sumet                    0.096363                    -0.167811   

                             DER_mass_vis  DER_pt_h  DER_deltar_tau_lep  \
DER_mass_MMC                     0.805154  0.035232            0.458074   
DER_mass_transverse_met_lep      0.190109 -0.249116            0.043251   
DER_mass_vis                     1.000000 -0.062562            0.579712   
DER_pt_h                        -0.062562  1.000000           -0.539379   
DER_deltar_tau_lep               0.579712 -0.539379            1.000000   
DER_pt_tot                      -0.000702  0.310501           -0.148081   
DER_sum_pt                       0.088685  0.832733           -0.432603   
DER_pt_ratio_lep_tau             0.097490  0.089187            0.047046   
DER_met_phi_centrality          -0.090846  0.539356           -0.205441   
PRI_tau_pt                       0.290011  0.407421           -0.202035   
PRI_lep_pt                       0.405482  0.360939           -0.069957   
PRI_met                         -0.087330  0.679585           -0.402345   
PRI_met_sumet                    0.053300  0.782547           -0.407002   

                             DER_pt_tot  DER_sum_pt  DER_pt_ratio_lep_tau  \
DER_mass_MMC                   0.015142    0.129819              0.064216   
DER_mass_transverse_met_lep    0.017758   -0.146837              0.349504   
DER_mass_vis                  -0.000702    0.088685              0.097490   
DER_pt_h                       0.310501    0.832733              0.089187   
DER_deltar_tau_lep            -0.148081   -0.432603              0.047046   
DER_pt_tot                     1.000000    0.381160              0.039193   
DER_sum_pt                     0.381160    1.000000              0.108791   
DER_pt_ratio_lep_tau           0.039193    0.108791              1.000000   
DER_met_phi_centrality         0.178448    0.420679             -0.038803   
PRI_tau_pt                     0.095754    0.485847             -0.474633   
PRI_lep_pt                     0.109617    0.460938              0.701142   
PRI_met                        0.269739    0.520129              0.053386   
PRI_met_sumet                  0.448925    0.904481              0.049904   

                             DER_met_phi_centrality  PRI_tau_pt  PRI_lep_pt  \
DER_mass_MMC                               0.041740    0.255751    0.321339   
DER_mass_transverse_met_lep               -0.419757   -0.145464    0.310648   
DER_mass_vis                              -0.090846    0.290011    0.405482   
DER_pt_h                                   0.539356    0.407421    0.360939   
DER_deltar_tau_lep                        -0.205441   -0.202035   -0.069957   
DER_pt_tot                                 0.178448    0.095754    0.109617   
DER_sum_pt                                 0.420679    0.485847    0.460938   
DER_pt_ratio_lep_tau                      -0.038803   -0.474633    0.701142   
DER_met_phi_centrality                     1.000000    0.140885    0.054047   
PRI_tau_pt                                 0.140885    1.000000    0.104

In [14]:
for column in corr_coef.columns:
    temp = corr_coef[abs(corr_coef[column]) > 0.78].index.values
    if np.where(temp != column)[0].shape[0] == 0:
        continue
    print(column)
    print(temp[np.where(temp != column)])
    print('\n')


DER_mass_MMC
['DER_mass_vis']


DER_mass_vis
['DER_mass_MMC']


DER_pt_h
['DER_sum_pt' 'PRI_met_sumet']


DER_sum_pt
['DER_pt_h' 'PRI_met_sumet']


PRI_met_sumet
['DER_pt_h' 'DER_sum_pt']




In [15]:
feature_0 = train[train['PRI_jet_num'] == 0]
feature_1 = train[train['PRI_jet_num'] == 1]
feature_2 = train[train['PRI_jet_num'] == 2]
feature_3 = train[train['PRI_jet_num'] == 3]

In [16]:
drop_feature_0 = ['DER_deltaeta_jet_jet',
                                     'DER_mass_jet_jet',
                                     'DER_prodeta_jet_jet',
                                     'DER_lep_eta_centrality',
                                     'PRI_jet_num',
                                     'PRI_jet_leading_pt',
                                     'PRI_jet_leading_eta',
                                     'PRI_jet_leading_phi',
                                     'PRI_jet_subleading_pt',
                                     'PRI_jet_subleading_eta',
                                     'PRI_jet_subleading_phi',
                                     'PRI_jet_all_pt',
                                     'DER_met_phi_centrality',
                                      'DER_pt_ratio_lep_tau',
                                      'DER_pt_tot',
                                      'DER_sum_pt',
                                      'PRI_lep_phi',
                                      'PRI_lep_pt',
                                      'PRI_met',
                                      'PRI_met_phi',
                                      'PRI_met_sumet',
                                      'PRI_tau_phi'
                                     ]

In [17]:
feature_0 = feature_0.drop(columns = drop_feature_0)

In [18]:
drop_feature_1 = ['DER_deltaeta_jet_jet',
                                     'DER_mass_jet_jet',
                                     'DER_prodeta_jet_jet',
                                     'DER_lep_eta_centrality',
                                     'PRI_jet_num',
                                     'PRI_jet_subleading_pt',
                                     'PRI_jet_subleading_eta',
                                     'PRI_jet_subleading_phi',
                                      'DER_met_phi_centrality',
                                      'DER_pt_ratio_lep_tau',
                                      'DER_pt_tot',
                                      'DER_sum_pt',
                                      'PRI_jet_leading_phi',
                                      'PRI_lep_phi',
                                      'PRI_lep_pt',
                                      'PRI_met',
                                      'PRI_met_phi',
                                      'PRI_met_sumet',
                                      'PRI_tau_phi'
                                      
                                     ]

In [19]:
feature_1 = feature_1.drop(columns = drop_feature_1)

In [20]:
drop_feature_2 = ['PRI_jet_num',
                                      'DER_lep_eta_centrality',
                                     'DER_met_phi_centrality',
                                     'DER_pt_ratio_lep_tau',
                                     'DER_pt_tot',
                                     'DER_sum_pt',
                                     'PRI_jet_leading_phi',
                                     'PRI_jet_subleading_phi',
                                     'PRI_jet_subleading_pt',
                                     'PRI_lep_phi',
                                     'PRI_lep_pt',
                                     'PRI_met',
                                      'PRI_met_phi',
                                      'PRI_met_sumet',
                                      'PRI_tau_phi'
                                     ]

In [21]:
feature_2 = feature_2.drop(columns = drop_feature_2)

In [22]:
drop_feature_3 = ['PRI_jet_num',
                                     'DER_lep_eta_centrality',
                                     'DER_met_phi_centrality',
                                     'DER_pt_h',
                                     'DER_pt_ratio_lep_tau',
                                     'DER_pt_tot',
                                     'DER_sum_pt',
                                     'PRI_jet_leading_phi',
                                     'PRI_jet_subleading_phi',
                                     'PRI_jet_subleading_pt',
                                     'PRI_lep_phi',
                                     'PRI_lep_pt',
                                     'PRI_met',
                                      'PRI_met_phi',
                                      'PRI_met_sumet',
                                      'PRI_tau_phi'
                                     ]

In [23]:
feature_3 = feature_3.drop(columns = drop_feature_3)

In [24]:
feature_0_corr = feature_0.drop(columns=['Prediction']).corr()
feature_1_corr = feature_1.drop(columns=['Prediction']).corr()
feature_2_corr = feature_2.drop(columns=['Prediction']).corr()
feature_3_corr = feature_3.drop(columns=['Prediction']).corr()


## feature_0

In [25]:
for column in feature_0_corr.columns:
    temp = feature_0_corr[abs(feature_0_corr[column]) > 0.8].index.values
    if np.where(temp != column)[0].shape[0] == 0:
        continue
    print(column)
    print(temp[np.where(temp != column)])
    print('\n')

In [26]:
feature_0.head(10)

Prediction  DER_mass_MMC  DER_mass_transverse_met_lep  DER_mass_vis  \
Id                                                                            
100003          -1       143.905                       81.417        80.943   
100004          -1       175.864                       16.915       134.805   
100008          -1       105.594                       50.559       100.989   
100010          -1      -999.000                       86.240        79.692   
100013          -1        82.488                       31.663        64.128   
100014          -1      -999.000                      109.412        14.398   
100015           1       111.026                       32.096        75.271   
100017           1       127.861                       50.953        77.267   
100018          -1      -999.000                       85.186        68.827   
100019          -1      -999.000                       88.767       115.058   

        DER_pt_h  DER_deltar_tau_lep  PRI_tau_pt  PRI_tau_eta  PRI_lep_eta  
Id                                                                          
100003     0.414               3.310      22.647       -1.655       -0.522  
100004    16.405               3.891      28.209       -2.197        0.798  
100008     4.288               2.904      39.008        2.433        0.210  
100010    27.201               2.338      29.718       -0.866        0.126  
100013     8.232               2.823      25.470       -0.654       -1.665  
100014    17.323               0.472      22.552        1.389        1.856  
100015    23.067               3.205      30.606       -1.107       -1.944  
100017    26.967               2.833      30.739       -0.635       -0.343  
100018     5.042               2.116      27.931        1.175        2.332  
100019    15.337               2.879      31.046        1.380       -1.486

In [27]:
for column in feature_0.columns[1:]:
    s_feature = feature_0[column][feature_0.index[(feature_0.Prediction == 1)]]
    if -999 in s_feature.values:
        print(column,'---s')
        median_0_s = np.median(s_feature[s_feature != -999])
        s_feature = s_feature.replace(to_replace = -999, value = median_0_s)
        feature_0.loc[feature_0.index[(feature_0.Prediction == 1)], (column)] = s_feature
        
    b_feature = feature_0[column][feature_0.index[(feature_0.Prediction == -1)]]
    if -999 in b_feature.values:
        print(column,'---b')
        median_0_b = np.median(b_feature[b_feature != -999])
        b_feature = b_feature.replace(to_replace = -999, value = median_0_b)
        feature_0.loc[feature_0.index[(feature_0.Prediction == -1)],(column)] = b_feature


DER_mass_MMC ---s
DER_mass_MMC ---b


In [28]:
for column in feature_0.columns[1:]:
    mean = np.mean(feature_0[column])
    std = np.std(feature_0[column])
    feature_0.loc[:, column] = (feature_0.loc[:, column] - mean) / std


## feature_1

In [29]:
for column in feature_1_corr.columns:
    temp = feature_1_corr[abs(feature_1_corr[column]) > 0.8].index.values
    if np.where(temp != column)[0].shape[0] == 0:
        continue
    print(column)
    print(temp[np.where(temp != column)])
    print('\n')

DER_pt_h
['PRI_jet_leading_pt' 'PRI_jet_all_pt']


PRI_jet_leading_pt
['DER_pt_h' 'PRI_jet_all_pt']


PRI_jet_all_pt
['DER_pt_h' 'PRI_jet_leading_pt']




In [30]:
feature_1.head(10)

Prediction  DER_mass_MMC  DER_mass_transverse_met_lep  DER_mass_vis  \
Id                                                                            
100001          -1       160.937                       68.768       103.235   
100002          -1      -999.000                      162.172       125.953   
100007           1       154.916                       10.418        94.714   
100009           1       128.053                       88.941        69.272   
100012          -1       145.297                       64.234       103.565   
100016          -1       114.256                        4.351        67.963   
100026           1       137.197                       68.009        78.296   
100028           1       118.104                        2.633        77.310   
100030          -1      -999.000                       91.316        51.771   
100034          -1       156.894                       39.256       122.894   

        DER_pt_h  DER_deltar_tau_lep  PRI_tau_pt  PRI_tau_eta  PRI_lep_eta  \
Id                                                                           
100001    48.146               3.473      42.014        2.039        0.501   
100002    35.635               3.148      32.154       -0.705       -0.953   
100007    29.169               2.897      78.800        0.654        0.506   
100009   193.392               1.609      54.646       -1.533       -0.317   
100012   106.999               2.183      62.890       -0.766        0.722   
100016    47.221               2.954      30.145        0.484       -0.215   
100026    35.332               2.883      34.657        1.587        1.664   
100028    91.388               1.976      37.999       -0.214       -1.354   
100030    33.742               1.309      29.267       -1.346       -2.443   
100034    71.682               2.387      48.605       -2.123       -0.456   

        PRI_jet_leading_pt  PRI_jet_leading_eta  PRI_jet_all_pt  
Id                                                               
100001              46.226                0.725          46.226  
100002              44.251                2.053          44.251  
100007              30.638               -0.715          30.638  
100009             167.735               -2.767         167.735  
100012              93.117               -0.970          93.117  
100016              36.263               -0.766          36.263  
100026              35.527                4.347          35.527  
100028              77.221               -0.049          77.221  
100030              33.250               -3.199          33.250  
100034              45.843                1.131          45.843

In [31]:
for column in feature_1.columns[1:]:
    s_feature = feature_1[column][feature_1.index[(feature_1.Prediction == 1)]]
    if -999 in s_feature.values:
        print(column,'---s')
        median_1_s = np.median(s_feature[s_feature != -999])
        s_feature = s_feature.replace(to_replace = -999, value = median_1_s)
        feature_1.loc[feature_1.index[(feature_1.Prediction == 1)], (column)] = s_feature
        
    b_feature = feature_1[column][feature_1.index[(feature_1.Prediction == -1)]]
    if -999 in b_feature.values:
        print(column,'---b')
        median_1_b = np.median(b_feature[b_feature != -999])
        b_feature = b_feature.replace(to_replace = -999, value = median_1_b)
        feature_1.loc[feature_1.index[(feature_1.Prediction == -1)],(column)] = b_feature


DER_mass_MMC ---s
DER_mass_MMC ---b


In [32]:
for column in feature_1.columns[1:]:
    mean = np.mean(feature_1[column])
    std = np.std(feature_1[column])
    feature_1.loc[:, column] = (feature_1.loc[:, column] - mean) / std


## feature_2

In [33]:
for column in feature_2_corr.columns:
    temp = feature_2_corr[abs(feature_2_corr[column]) > 0.8].index.values
    if np.where(temp != column)[0].shape[0] == 0:
        continue
    print(column)
    print(temp[np.where(temp != column)])
    print('\n')

DER_deltaeta_jet_jet
['DER_mass_jet_jet' 'DER_prodeta_jet_jet']


DER_mass_jet_jet
['DER_deltaeta_jet_jet']


DER_prodeta_jet_jet
['DER_deltaeta_jet_jet']


PRI_jet_leading_pt
['PRI_jet_all_pt']


PRI_jet_all_pt
['PRI_jet_leading_pt']




In [34]:
for column in feature_2.columns[1:]:
    s_feature = feature_2[column][feature_2.index[(feature_2.Prediction == 1)]]
    if -999 in s_feature.values:
        print(column,'---s')
        median_2_s = np.median(s_feature[s_feature != -999])
        s_feature = s_feature.replace(to_replace = -999, value = median_2_s)
        feature_2.loc[feature_2.index[(feature_2.Prediction == 1)], (column)] = s_feature
        
    b_feature = feature_2[column][feature_2.index[(feature_2.Prediction == -1)]]
    if -999 in b_feature.values:
        print(column,'---b')
        median_2_b = np.median(b_feature[b_feature != -999])
        b_feature = b_feature.replace(to_replace = -999, value = median_1_b)
        feature_2.loc[feature_2.index[(feature_2.Prediction == -1)],(column)] = b_feature

DER_mass_MMC ---s
DER_mass_MMC ---b


In [35]:
for column in feature_2.columns[1:]:
    mean = np.mean(feature_2[column])
    std = np.std(feature_2[column])
    feature_2.loc[:, column] = (feature_2.loc[:, column] - mean) / std


## feature_3

In [36]:
for column in feature_3_corr.columns:
    temp = feature_3_corr[abs(feature_3_corr[column]) > 0.8].index.values
    if np.where(temp != column)[0].shape[0] == 0:
        continue
    print(column)
    print(temp[np.where(temp != column)])
    print('\n')

PRI_jet_leading_pt
['PRI_jet_all_pt']


PRI_jet_all_pt
['PRI_jet_leading_pt']




In [37]:
for column in feature_3.columns[1:]:
    s_feature = feature_3[column][feature_3.index[(feature_3.Prediction == 1)]]
    if -999 in s_feature.values:
        print(column,'---s')
        median_3_s = np.median(s_feature[s_feature != -999])
        s_feature = s_feature.replace(to_replace = -999, value = median_3_s)
        feature_3.loc[feature_3.index[(feature_3.Prediction == 1)], (column)] = s_feature
        
    b_feature = feature_3[column][feature_3.index[(feature_3.Prediction == -1)]]
    if -999 in b_feature.values:
        print(column,'---b')
        median_3_b = np.median(b_feature[b_feature != -999])
        b_feature = b_feature.replace(to_replace = -999, value = median_3_b)
        feature_3.loc[feature_3.index[(feature_3.Prediction == -1)],(column)] = b_feature

DER_mass_MMC ---s
DER_mass_MMC ---b


In [38]:
for column in feature_3.columns[1:]:
    mean = np.mean(feature_3[column])
    std = np.std(feature_3[column])
    feature_3.loc[:, column] = (feature_3.loc[:, column] - mean) / std


In [41]:
feature_3.head(10)

Prediction  DER_mass_MMC  DER_mass_transverse_met_lep  DER_mass_vis  \
Id                                                                            
100005          -1     -0.463036                    -0.750383     -0.405341   
100011          -1     -0.099534                    -0.836110     -0.065865   
100031           1      0.001330                    -0.947603     -0.117310   
100038           1      0.243873                    -0.107322      0.412227   
100039           1     -0.015755                    -0.509301      0.204212   
100059          -1     -0.694586                     1.093045     -0.374392   
100060           1     -0.260900                    -0.031575     -0.300914   
100070           1     -0.019433                    -0.314549      0.066622   
100077          -1     -0.508328                    -0.217055     -0.234179   
100082           1      0.655037                    -0.689501      0.031799   

        DER_deltaeta_jet_jet  DER_mass_jet_jet  DER_prodeta_jet_jet  \
Id                                                                    
100005              0.471619         -0.134256            -0.134641   
100011              0.421956         -0.235069            -0.434927   
100031              0.473660         -0.098458            -0.552460   
100038             -0.087605         -0.163011             0.578930   
100039             -1.173399         -0.504313             1.856800   
100059             -0.305308         -0.520022            -0.136734   
100060             -0.859090         -0.800964             0.628803   
100070              1.343792          0.253407            -1.151985   
100077              0.704289         -0.308945            -0.528744   
100082             -0.250202         -0.353194            -0.136036   

        DER_deltar_tau_lep  PRI_tau_pt  PRI_tau_eta  PRI_lep_eta  \
Id                                                                 
100005           -0.631763    0.212150     0.311179    -0.726206   
100011            1.212923   -0.332868    -0.552099    -0.135727   
100031           -0.890454    2.445260     0.794283     0.189488   
100038            0.870822   -0.784608     1.214302    -0.930697   
100039            0.701585   -0.236322    -0.894090    -0.973402   
100059           -0.373072   -0.517141     1.679974     1.564260   
100060           -0.435931    0.442276    -0.490674    -1.148328   
100070            0.805545   -0.211160     1.632660     0.963927   
100077            0.080242   -0.562099    -0.391895    -1.744555   
100082            1.192373   -0.226023    -0.528027     0.651031   

        PRI_jet_leading_pt  PRI_jet_leading_eta  PRI_jet_subleading_eta  \
Id                                                                        
100005           -0.426680            -1.524375                0.130561   
100011           -0.607176            -0.493374                1.003083   
100031            0.777613             0.887862               -0.703658   
100038           -0.111461             0.402237                1.374848   
100039           -0.454827            -1.369280               -1.339038   
100059           -0.564338            -0.535326                0.363759   
100060           -0.588817            -1.051462               -0.552698   
100070           -0.834820             0.861801               -1.447188   
100077           -0.866925             0.484869               -1.251166   
100082           -0.494231            -0.666267                0.293349   

        PRI_jet_all_pt  
Id                      
100005       -0.480030  
100011       -0.698167  
100031       -0.014476  
100038        0.674392  
100039       -0.441322  
100059       -0.567588  
100060       -0.334807  
100070       -0.722012  
100077       -0.897308  
100082        0.689269

## Logistic Regression

In [39]:
clf_0 = LogisticRegressionCV(max_iter = 1000, cv = 4,n_jobs = -1, verbose=1).fit(feature_0.drop(columns='Prediction'), feature_0['Prediction'])
clf_1 = LogisticRegressionCV(max_iter = 1000, cv = 4,n_jobs = -1, verbose=1).fit(feature_1.drop(columns='Prediction'), feature_1['Prediction'])
clf_2 = LogisticRegressionCV(max_iter = 1000, cv = 4,n_jobs = -1, verbose=1).fit(feature_2.drop(columns='Prediction'), feature_2['Prediction'])
clf_3 = LogisticRegressionCV(max_iter = 1000, cv = 4,n_jobs = -1, verbose=1).fit(feature_3.drop(columns='Prediction'), feature_3['Prediction'])






[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.2s finished


In [40]:
print(clf_0.score(feature_0.drop(columns='Prediction'), feature_0['Prediction']))
print(clf_1.score(feature_1.drop(columns='Prediction'), feature_1['Prediction']))
print(clf_2.score(feature_2.drop(columns='Prediction'), feature_2['Prediction']))
print(clf_3.score(feature_3.drop(columns='Prediction'), feature_3['Prediction']))



0.821464674266612
0.7099840090787166
0.722007185533655
0.7101606208265656


In [59]:
test = pd.read_csv('test.csv', index_col= 0)

In [60]:
test_0 = test[test['PRI_jet_num'] == 0]
test_1 = test[test['PRI_jet_num'] == 1]
test_2 = test[test['PRI_jet_num'] == 2]
test_3 = test[test['PRI_jet_num'] == 3]

In [61]:
median_mass_MMC = np.median(test_0.DER_mass_MMC[test_0.DER_mass_MMC != -999])
test_0.loc[:,'DER_mass_MMC'] = test_0.loc[:,'DER_mass_MMC'].replace({-999: median_mass_MMC})

median_mass_MMC = np.median(test_1.DER_mass_MMC[test_1.DER_mass_MMC != -999])
test_1.loc[:,'DER_mass_MMC'] = test_1.loc[:,'DER_mass_MMC'].replace({-999: median_mass_MMC})

median_mass_MMC = np.median(test_2.DER_mass_MMC[test_2.DER_mass_MMC != -999])
test_2.loc[:,'DER_mass_MMC'] = test_2.loc[:,'DER_mass_MMC'].replace({-999: median_mass_MMC})

median_mass_MMC = np.median(test_3.DER_mass_MMC[test_3.DER_mass_MMC != -999])
test_3.loc[:,'DER_mass_MMC'] = test_3.loc[:,'DER_mass_MMC'].replace({-999: median_mass_MMC})

/Users/wentao/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [62]:
test_3.head(10)

Prediction  DER_mass_MMC  DER_mass_transverse_met_lep  DER_mass_vis  \
Id                                                                           
350004          ?        74.159                       82.772        58.731   
350022          ?        80.256                       36.728        48.894   
350025          ?       103.413                       23.534        52.474   
350028          ?       128.683                       86.650        81.667   
350035          ?       107.684                       76.191        51.874   
350071          ?       325.980                      101.905       260.669   
350085          ?       147.815                       20.427        87.309   
350099          ?        94.060                      129.002        86.738   
350118          ?       145.641                       42.595        85.479   
350131          ?       127.031                       50.363        83.770   

        DER_pt_h  DER_deltaeta_jet_jet  DER_mass_jet_jet  DER_prodeta_jet_jet  \
Id                                                                              
350004    89.646                 1.347           536.663               -0.339   
350022    83.782                 1.205           285.153                0.078   
350025   171.495                 0.975            84.789               -0.219   
350028    37.521                 4.337           836.374               -3.209   
350035    70.051                 0.601            36.783                1.358   
350071    73.574                 0.586            87.810                0.263   
350085    49.791                 2.568           358.760                2.478   
350099    75.573                 1.302           252.886                0.376   
350118    99.728                 1.401           273.234                2.126   
350131   183.100                 0.559            93.389                1.342   

        DER_deltar_tau_lep  DER_pt_tot       ...        PRI_met_phi  \
Id                                           ...                      
350004               1.028      77.213       ...             -0.231   
350022               1.965     148.636       ...             -2.722   
350025               1.296      49.861       ...              1.376   
350028               2.554     109.694       ...             -0.930   
350035               1.070      74.854       ...              0.731   
350071               3.986      85.763       ...              0.055   
350085               2.765      25.002       ...             -1.634   
350099               1.429      39.819       ...             -1.333   
350118               2.715      46.481       ...             -2.870   
350131               1.874      40.429       ...             -3.095   

        PRI_met_sumet  PRI_jet_num  PRI_jet_leading_pt  PRI_jet_leading_eta  \
Id                                                                            
350004        869.614            3             254.085               -1.013   
350022        561.120            3             181.886                0.062   
350025        277.138            3              71.811                0.352   
350028        571.086            3             121.317               -0.946   
350035        360.034            3              49.252               -0.903   
350071        321.176            3             102.086                0.298   
350085        372.161            3             121.289                0.747   
350099        526.980            3             137.030                0.243   
350118        469.051            3             143.298                0.917   
350131        385.110            3             147.437               -0.912   

        PRI_jet_leading_phi  PRI_jet_subleading_pt  PRI_jet_subleading_eta  \
Id                                                                           
350004               -0.334                185.857                   0.335   
350022                2.115                 78.088           

In [63]:
test_0 = test_0.drop(columns = drop_feature_0)
test_1 = test_1.drop(columns = drop_feature_1)
test_2 = test_2.drop(columns = drop_feature_2)
test_3 = test_3.drop(columns = drop_feature_3)

In [68]:
test_3.columns

Index(['Prediction', 'DER_mass_MMC', 'DER_mass_transverse_met_lep',
       'DER_mass_vis', 'DER_deltaeta_jet_jet', 'DER_mass_jet_jet',
       'DER_prodeta_jet_jet', 'DER_deltar_tau_lep', 'PRI_tau_pt',
       'PRI_tau_eta', 'PRI_lep_eta', 'PRI_jet_leading_pt',
       'PRI_jet_leading_eta', 'PRI_jet_subleading_eta', 'PRI_jet_all_pt'],
      dtype='object')

In [69]:
for column in test_0.columns[1:]:
    mean = np.mean(test_0[column])
    std = np.std(test_0[column])
    test_0.loc[:, column] = (test_0.loc[:, column] - mean) / std

for column in test_1.columns[1:]:
    mean = np.mean(test_1[column])
    std = np.std(test_1[column])
    test_1.loc[:, column] = (test_1.loc[:, column] - mean) / std

for column in test_2.columns[1:]:
    mean = np.mean(test_2[column])
    std = np.std(test_2[column])
    test_2.loc[:, column] = (test_2.loc[:, column] - mean) / std

for column in test_3.columns[1:]:
    mean = np.mean(test_3[column])
    std = np.std(test_3[column])
    test_3.loc[:, column] = (test_3.loc[:, column] - mean) / std

In [70]:
test_0.head(10)

Prediction  DER_mass_MMC  DER_mass_transverse_met_lep  DER_mass_vis  \
Id                                                                           
350000          ?     -0.151047                     0.649975     -1.552406   
350002          ?     -0.008336                    -0.078977      0.396833   
350003          ?      0.400408                    -0.878411      0.421857   
350005          ?     -0.507983                     1.104856     -0.412493   
350008          ?     -1.067945                     0.236248     -0.555911   
350011          ?      0.563933                     1.448910      0.678663   
350012          ?     -0.500653                    -0.488023     -0.601115   
350014          ?     -0.151047                     0.641301     -1.782923   
350015          ?     -0.151047                     0.447979     -0.967004   
350016          ?      0.702911                     0.774904      0.044613   

        DER_pt_h  DER_deltar_tau_lep  PRI_tau_pt  PRI_tau_eta  PRI_lep_eta  
Id                                                                          
350000 -0.766918           -2.554639   -0.796086    -0.420343    -0.350282  
350002 -0.689181            0.130053    0.853151    -0.223802    -0.383880  
350003 -0.338479            0.211232    1.201987     0.333334    -0.190690  
350005  0.057976            0.579435   -0.918182    -0.966109    -0.446494  
350008 -0.927760           -0.428050   -0.554320     1.247006     0.898960  
350011  0.653187           -0.078692    0.152448     1.019603     1.499145  
350012 -0.618292            0.675110   -0.795951    -0.783375    -0.332719  
350014  0.266829           -3.064905   -0.570087    -1.109048    -0.654956  
350015 -0.729709           -1.292022   -0.733016    -0.471508    -0.018881  
350016 -0.868027            0.683808   -0.390110     1.277055     1.643465

In [71]:
predict_label_0 = clf_0.predict(test_0.drop(columns='Prediction'))
predict_label_1 = clf_1.predict(test_1.drop(columns='Prediction'))
predict_label_2 = clf_2.predict(test_2.drop(columns='Prediction'))
predict_label_3 = clf_3.predict(test_3.drop(columns='Prediction'))


In [72]:
np.sum(abs(predict_label_3))

50794

In [73]:
sub = pd.read_csv('sample-submission.csv', index_col=0)

In [74]:
sub.head()

Prediction
Id                
350000          -1
350001          -1
350002          -1
350003          -1
350004          -1

In [75]:
sub.loc[test_0.index,'Prediction'] = predict_label_0
sub.loc[test_1.index,'Prediction'] = predict_label_1
sub.loc[test_2.index,'Prediction'] = predict_label_2
sub.loc[test_3.index,'Prediction'] = predict_label_3

In [76]:
pd.DataFrame.to_csv(sub, 'sub.csv')